In [ ]:
import xarray as xr
import numpy as np
import zarr

In [ ]:
ncar_data = xr.open_dataset("/Volumes/backup-hdd/jhd.100.nc")
ncar_data.info

In [ ]:
# %matplotlib notebook
from matplotlib import pyplot as plt

# Turn interactive plotting off
plt.ioff() # Don't display plots until calling plt.show() - Faster

# plt.gray()

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
p = np.array(ncar_data['p'][:200])

In [ ]:
v = np.array(ncar_data['v'][:200])

In [ ]:
p.shape

In [ ]:
vmax = v.max() # This takes several seconds - preload
vmax

In [ ]:
vmin = v.min()
vmin

In [ ]:
plt.imshow(v[0], cmap='hot')

In [ ]:
emax = e.max() # This takes several seconds - preload
emin = e.min()

In [ ]:
def save_plots_sequential(array_name, val_min, val_max, nr_images):


    for i in range(200):
        plt.figure(figsize=(18,18))
        # vmin, vmax necessary for image brightness to stay consistent across frames
        plt.imshow(array_name[i], cmap='gray', vmin=val_min, vmax=val_max)

        plt.savefig("visualizations/" + str(i) + ".jpg")
        #, *, dpi='figure', format=None, metadata=None,
    #         bbox_inches=None, pad_inches=0.1,
    #         facecolor='auto', edgecolor='auto',
    #         backend=None, **kwargs
    #        )

In [ ]:
save_plots_sequential(v, vmin, vmax, 200)

In [ ]:
from joblib import Parallel, delayed

### <font color="orange">This says "Disk Quota exceeded" even though its not</font>

In [ ]:
def save_plots_parallel(array_name, val_min, val_max, i):
    # Turn interactive plotting off
    plt.ioff() # Don't display plots until calling plt.show() - Faster

    plt.figure(figsize=(18,18))
    # vmin, vmax necessary for image brightness to stay consistent across frames
    plt.imshow(a[i], cmap='gray', vmin=val_min, vmax=val_max)

    plt.savefig("visualizations/" + str(i) + ".jpg")
    #, *, dpi='figure', format=None, metadata=None,
#         bbox_inches=None, pad_inches=0.1,
#         facecolor='auto', edgecolor='auto',
#         backend=None, **kwargs
#        )

In [ ]:
_ = Parallel(n_jobs=8, backend="multiprocessing")(delayed(save_plots_parallel)(e, emin, emax, i) for i in range(200))

### Try plotting with FuncAnimation - <font color="orange">saving to MP4 runs forever when I last tried :(</font>

In [ ]:
%matplotlib widget

In [ ]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# data = np.random.rand(100,50,50)

fig, ax = plt.subplots()

var = "e"

# ax.set_xlim((0, 50))
# ax.set_ylim((0, 50))

im = ax.imshow(ds[var][:,:,0])

def init():
    im.set_data(ds[var][:,:,0])
    return (im,)

# animation function. This is called sequentially
def animate(i):
    data_slice = ds[var][:,:,i]
    im.set_data(data_slice)
    return (im,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=25, interval=50, blit=False)

HTML(anim.to_html5_video())

In [ ]:
# f = r"animation.gif"  # This runs forever
# writergif = animation.PillowWriter(fps=30) 
# anim.save(f, writer=writergif)

In [ ]:
from matplotlib.animation import FFMpegWriter

class FasterFFMpegWriter(FFMpegWriter):
    '''FFMpeg-pipe writer bypassing figure.savefig.'''
    def __init__(self, **kwargs):
        '''Initialize the Writer object and sets the default frame_format.'''
        super().__init__(**kwargs)
        self.frame_format = 'argb'

    def grab_frame(self, **savefig_kwargs):
        '''Grab the image information from the figure and save as a movie frame.

        Doesn't use savefig to be faster: savefig_kwargs will be ignored.
        '''
        try:
            # re-adjust the figure size and dpi in case it has been changed by the
            # user.  We must ensure that every frame is the same size or
            # the movie will not save correctly.
            self.fig.set_size_inches(self._w, self._h)
            self.fig.set_dpi(self.dpi)
            # Draw and save the frame as an argb string to the pipe sink
            self.fig.canvas.draw()
            self._proc.stdin.write(self.fig.canvas.tostring_argb())
        except (RuntimeError, IOError) as e:
            out, err = self._proc.communicate()
            raise IOError('Error saving animation to file (cause: {0}) '
                      'Stdout: {1} StdError: {2}. It may help to re-run '
                      'with --verbose-debug.'.format(e, out, err)) 

In [ ]:
writer = FasterFFMpegWriter()

In [ ]:
anim.save('im.mp4', writer=writer)

In [ ]:
# HTML(anim.to_html5_video())